## Setting

In [487]:
import pandas as pd
import numpy as np
import datetime 
import seaborn as sns
import glob

In [488]:
df= pd.read_csv('master_final_220708.csv')
# df= df.drop(columns="Unnamed: 0",axis=1)
df.head()

,cartype,예정시간,배차시간,승차시간,startpos1,startpos2,endpos1,endpos2,direction,start_lat,...,end_lng,distance,duration,taxiFare,y_b_time,b_s_time,y_s_time,y_b_seconds,b_s_seconds,y_s_seconds
0,중형승합,2018-01-23 00:04:00,2018-01-23 00:18:42,2018-01-23 00:34:08,성북구,장위제1동,동대문구,회기동,장위제1동회기동,127.043713,...,37.590808,4289,648.464,5600,0 days 00:14:42,0 days 00:15:26,0 days 00:30:08,882,926,1808
1,중형승합,2018-01-23 00:17:00,2018-01-23 00:50:37,2018-01-23 01:19:22,중랑구,면목제7동,노원구,상계1동,면목제7동상계1동,127.086985,...,37.679901,14493,1318.675,13700,0 days 00:33:37,0 days 00:28:45,0 days 01:02:22,2017,1725,3742
2,중형승합,2018-01-23 00:17:06,2018-01-23 00:17:39,2018-01-23 00:30:00,성북구,정릉제3동,강북구,수유제3동,정릉제3동수유제3동,127.004186,...,37.638715,4869,864.161,6200,0 days 00:00:33,0 days 00:12:21,0 days 00:12:54,33,741,774
3,중형승합,2018-01-23 00:47:00,2018-01-23 00:51:19,2018-01-23 01:09:33,구로구,고척제2동,관악구,은천동,고척제2동은천동,126.858712,...,37.485309,10247,1652.785,11000,0 days 00:04:19,0 days 00:18:14,0 days 00:22:33,259,1094,1353
4,중형승합,2018-01-23 01:08:00,2018-01-23 01:44:17,2018-01-23 02:06:53,동대문구,회기동,성북구,장위제2동,회기동장위제2동,127.055313,...,37.612390,3778,598.639,5200,0 days 00:36:17,0 days 00:22:36,0 days 00:58:53,2177,1356,3533


In [489]:
df.shape

(407553, 22)

In [490]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 407553 entries, 0 to 407552
Data columns (total 22 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   cartype      407553 non-null  object 
 1   예정시간         407553 non-null  object 
 2   배차시간         407553 non-null  object 
 3   승차시간         407553 non-null  object 
 4   startpos1    407553 non-null  object 
 5   startpos2    407553 non-null  object 
 6   endpos1      407553 non-null  object 
 7   endpos2      407553 non-null  object 
 8   direction    407553 non-null  object 
 9   start_lat    407553 non-null  float64
 10  start_lng    407553 non-null  float64
 11  end_lat      407553 non-null  float64
 12  end_lng      407553 non-null  float64
 13  distance     407553 non-null  int64  
 14  duration     407553 non-null  float64
 15  taxiFare     407553 non-null  int64  
 16  y_b_time     407553 non-null  object 
 17  b_s_time     407553 non-null  object 
 18  y_s_time     407553 non-

In [491]:
df[['distance','duration','taxiFare']].describe()

,distance,duration,taxiFare
count,407553.000000,407553.000000,407553.000000
mean,11227.803434,1427.078149,12017.347412
std,9384.646511,957.381900,8368.638228
min,205.000000,43.601000,0.000000
25%,4117.000000,679.163000,5730.000000
50%,8085.000000,1194.102000,9500.000000
75%,16139.000000,1918.202000,16300.000000
max,84449.000000,6880.172000,103750.000000


### 아이엠 콜택시 요금 산정
- 기본요금: 1000m(1km) 4000원 | 심야: 5000원 
- 거리요금: 120m(0.12km) 당 100원 
- 시간요금: 30초(0.5분) 당 100원 
- 기타요금: 시간거리와 완전 동시 병산▼
1) 심야할증 : 40% (0-4시)
2) 탄력요금제 : 0.8~4배 적용 -- 심야할증과 중복적용, 플랫폼 호출 고객 한정
3) 호출수수료 : 0~3000원

In [492]:
# 기본요금을 구합니다.

In [493]:
df["승차시간"]=pd.to_datetime(df["승차시간"])
df['승차시간_h'] = df['승차시간'].dt.hour
df['승차시간_h'].sort_values().unique()

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 23])

In [494]:
%%time 
기본_val = []

for i in range(len(df)):
    if df['승차시간_h'][i] in (0,1,2,3,4): ##심야시간 지정
        기본_cal = 5000
        기본_val.append(기본_cal) 
    else:
        기본_cal = 4000
        기본_val.append(기본_cal)
        
df['기본요금_아'] = 기본_val

CPU times: user 1.63 s, sys: 7.87 ms, total: 1.64 s
Wall time: 1.64 s


In [495]:
# 거리요금을 구합니다↓.--기본단위:m

In [496]:
%%time 
거리_val = []

for i in range(len(df)):
    if df['distance'][i]>1000:
        거리_cal = ((df['distance'][i]-1000)/120)*100
        거리_val.append(거리_cal)
    else:
        거리_val.append(0)
        
df['거리요금_아'] = 거리_val

CPU times: user 2.91 s, sys: 5.18 ms, total: 2.91 s
Wall time: 2.91 s


In [497]:
# 시간요금을 구합니다↓.--기본단위:초 

In [498]:
%%time 
시간_val = []

for i in range(len(df)):
    if df['duration'][i]>240 & df['승차시간_h'][i] in (0,1,2,3,4): ## 1Km 4분 제외한 추가 시간요금 &야간할증40%
        시간_cal = ((df['duration'][i]-240)/30)*100 + ( ((df['duration'][i]-240)/30)*100 )*0.4
        시간_val.append(시간_cal)
        
    elif df['duration'][i]>240: ## 1Km 4분 제외한 추가 시간요금 & 할증 NO
        시간_cal = ((df['duration'][i]-240)/30)*100
        시간_val.append(시간_cal)
        
    else: ## 4분 이하
        시간_val.append(0)
        
df['시간요금_아'] = 시간_val

CPU times: user 6 s, sys: 10 ms, total: 6.01 s
Wall time: 6.01 s


In [499]:
df.iloc[:,-7:].head()

,y_b_seconds,b_s_seconds,y_s_seconds,승차시간_h,기본요금_아,거리요금_아,시간요금_아
0,882,926,1808,0,5000,2740.833333,1906.165333
1,2017,1725,3742,1,5000,11244.166667,5033.816667
2,33,741,774,0,5000,3224.166667,2912.751333
3,259,1094,1353,1,5000,7705.833333,6592.996667
4,2177,1356,3533,2,5000,2315.000000,1673.648667


In [500]:
df[df['taxiFare']==0].iloc[:,-7:].head() ##거리가 1Km일떄 taxiFare=0-> 기본요금만 받는다.

,y_b_seconds,b_s_seconds,y_s_seconds,승차시간_h,기본요금_아,거리요금_아,시간요금_아
17,892,441,1333,3,5000,0.0,0.0
35,384,1155,1539,5,4000,0.0,0.0
234,255,1066,1321,7,4000,0.0,0.0
301,5115,898,6013,7,4000,0.0,0.0
361,1874,1061,2935,7,4000,0.0,0.0


In [501]:
# 기타요금을 구합니다↓

- 0-4시 : 심야 할증 +40%
- 5-6시 : 탄력 0.8배
- 7-9시: 탄력 1.2배
- 10-나머지 : 기존 변동 없음

In [502]:
df['taxiFare_아'] = (df['기본요금_아']+ df['거리요금_아']) 

In [503]:
df.iloc[:,-10:]

,b_s_time,y_s_time,y_b_seconds,b_s_seconds,y_s_seconds,승차시간_h,기본요금_아,거리요금_아,시간요금_아,taxiFare_아
0,0 days 00:15:26,0 days 00:30:08,882,926,1808,0,5000,2740.833333,1906.165333,7740.833333
1,0 days 00:28:45,0 days 01:02:22,2017,1725,3742,1,5000,11244.166667,5033.816667,16244.166667
2,0 days 00:12:21,0 days 00:12:54,33,741,774,0,5000,3224.166667,2912.751333,8224.166667
3,0 days 00:18:14,0 days 00:22:33,259,1094,1353,1,5000,7705.833333,6592.996667,12705.833333
4,0 days 00:22:36,0 days 00:58:53,2177,1356,3533,2,5000,2315.000000,1673.648667,7315.000000
...,...,...,...,...,...,...,...,...,...,...
407548,0 days 00:42:46,0 days 00:55:24,758,2566,3324,7,4000,12143.333333,15593.223333,16143.333333
407549,0 days 00:16:25,0 days 00:25:35,550,985,1535,7,4000,0.000000,0.000000,4000.000000
407550,0 days 00:21:08,0 days 00:25:35,267,1268,1535,7,4000,38606.666667,11326.009333,42606.666667
407551,0 days 01:06:22,0 days 01:11:21,299,3982,4281,8,4000,3025.833333,2487.128000,7025.833333


In [504]:
기타포함총금액_val = []

for i in range(len(df)):
    if df['승차시간_h'][i] in (5,6): ## 5-6시 : 탄력 0.8배
        기타포함총금액_cal = df['taxiFare_아'][i]*0.8
        기타포함총금액_val.append(기타포함총금액_cal)
        
    elif df['승차시간_h'][i] in (7,8,9): ## 7-9시 : 탄력 1.2배
        기타포함총금액_cal = df['taxiFare_아'][i]*1.2
        기타포함총금액_val.append(기타포함총금액_cal)  
    else: ## 나머지 기존 동일
        기타포함총금액_cal = df['taxiFare_아'][i]
        기타포함총금액_val.append(기타포함총금액_cal) 

df['taxiFare_아2'] = 기타포함총금액_val

## 100원 이하 절사
def aa(x):
    pd.to_numeric(x) 
    x=math.trunc(x/100)*100
    return x
df['taxiFare_아'] = df['taxiFare_아'].apply(aa) 
df['taxiFare_아2'] = df['taxiFare_아2'].apply(aa) 

In [505]:
df.iloc[:,-10:]

,y_s_time,y_b_seconds,b_s_seconds,y_s_seconds,승차시간_h,기본요금_아,거리요금_아,시간요금_아,taxiFare_아,taxiFare_아2
0,0 days 00:30:08,882,926,1808,0,5000,2740.833333,1906.165333,7700,7700
1,0 days 01:02:22,2017,1725,3742,1,5000,11244.166667,5033.816667,16200,16200
2,0 days 00:12:54,33,741,774,0,5000,3224.166667,2912.751333,8200,8200
3,0 days 00:22:33,259,1094,1353,1,5000,7705.833333,6592.996667,12700,12700
4,0 days 00:58:53,2177,1356,3533,2,5000,2315.000000,1673.648667,7300,7300
...,...,...,...,...,...,...,...,...,...,...
407548,0 days 00:55:24,758,2566,3324,7,4000,12143.333333,15593.223333,16100,19300
407549,0 days 00:25:35,550,985,1535,7,4000,0.000000,0.000000,4000,4800
407550,0 days 00:25:35,267,1268,1535,7,4000,38606.666667,11326.009333,42600,51100
407551,0 days 01:11:21,299,3982,4281,8,4000,3025.833333,2487.128000,7000,8400


### 서울시 장애인 콜택시 요금 산정
- 기본요금: 5000m(5km) 1500원  
- 거리요금: 1000m(1km) 당 280원 | 10000m(10Km) 도달시 2900원 -> 10km 초과시: 1000m(1km) 당 70원
- 시간요금: x
- 기타요금: 지역간 할증x

In [506]:
# 기본요금,거리요금, 총 이용금액을 구합니다. 
import math

In [507]:
%%time
기본_val = []
거리_val = []
taxiFare_val = []

for i in range(len(df)):
    if df['distance'][i] < 5000:
        기본_cal = 1500
        거리_cal = 0
        기본_val.append(기본_cal)
        거리_val.append(거리_cal)
    elif 5000 <= df['distance'][i] < 10000:
        기본_cal = 1500
        거리_cal = ((df['distance'][i]-5000)/1000)*280
        기본_val.append(기본_cal)
        거리_val.append(거리_cal)
    elif df['distance'][i] == 10000:
        기본_cal = 2900
        거리_cal = 0 ## 10Km 원래 계산은 1500 + (280*5)= 3400원. but 10km 도달시 2900원 디폴트로 본다. 
        기본_val.append(기본_cal)
        거리_val.append(거리_cal)
    elif df['distance'][i] >= 10000:
        기본_cal = 2900
        거리_cal = ((df['distance'][i]-10000)/1000)*70
        기본_val.append(기본_cal)
        거리_val.append(거리_cal)
    else:
        기본_cal = 999 ##오류점검용
        거리_cal = 999
        기본_val.append(기본_cal)
        거리_val.append(거리_cal)
        
df['기본요금_장']= 기본_val
df['거리요금_장']= 거리_val 
df['taxiFare_장'] = (df['기본요금_장']+ df['거리요금_장']) 

## 100원 이하 절사
def aa(x):
    pd.to_numeric(x) 
    x=math.trunc(x/100)*100
    return x
df['taxiFare_장'] = df['taxiFare_장'].apply(aa) 

CPU times: user 5.24 s, sys: 11.8 ms, total: 5.25 s
Wall time: 5.25 s


In [508]:
df.iloc[:,-7:]

,거리요금_아,시간요금_아,taxiFare_아,taxiFare_아2,기본요금_장,거리요금_장,taxiFare_장
0,2740.833333,1906.165333,7700,7700,1500,0.00,1500
1,11244.166667,5033.816667,16200,16200,2900,314.51,3200
2,3224.166667,2912.751333,8200,8200,1500,0.00,1500
3,7705.833333,6592.996667,12700,12700,2900,17.29,2900
4,2315.000000,1673.648667,7300,7300,1500,0.00,1500
...,...,...,...,...,...,...,...
407548,12143.333333,15593.223333,16100,19300,2900,390.04,3200
407549,0.000000,0.000000,4000,4800,1500,0.00,1500
407550,38606.666667,11326.009333,42600,51100,2900,2612.96,5500
407551,3025.833333,2487.128000,7000,8400,1500,0.00,1500


In [509]:
df[df['기본요금_장']==999].iloc[:,-7:] ##오류없다

,거리요금_아,시간요금_아,taxiFare_아,taxiFare_아2,기본요금_장,거리요금_장,taxiFare_장


In [510]:
# df.groupby(['taxiFare_장'])['기본요금_장','거리요금_장','taxiFare_장'].agg('nunique').reset_index()

In [511]:
df.iloc[:,-10:]

,y_s_seconds,승차시간_h,기본요금_아,거리요금_아,시간요금_아,taxiFare_아,taxiFare_아2,기본요금_장,거리요금_장,taxiFare_장
0,1808,0,5000,2740.833333,1906.165333,7700,7700,1500,0.00,1500
1,3742,1,5000,11244.166667,5033.816667,16200,16200,2900,314.51,3200
2,774,0,5000,3224.166667,2912.751333,8200,8200,1500,0.00,1500
3,1353,1,5000,7705.833333,6592.996667,12700,12700,2900,17.29,2900
4,3533,2,5000,2315.000000,1673.648667,7300,7300,1500,0.00,1500
...,...,...,...,...,...,...,...,...,...,...
407548,3324,7,4000,12143.333333,15593.223333,16100,19300,2900,390.04,3200
407549,1535,7,4000,0.000000,0.000000,4000,4800,1500,0.00,1500
407550,1535,7,4000,38606.666667,11326.009333,42600,51100,2900,2612.96,5500
407551,4281,8,4000,3025.833333,2487.128000,7000,8400,1500,0.00,1500


In [512]:
df = df.drop(['승차시간_h','기본요금_아','거리요금_아','시간요금_아','taxiFare_아','기본요금_장','거리요금_장'],axis=1)

In [513]:
df.rename(columns = {'taxiFare':'taxiFare_N','taxiFare_아2':'taxiFare_I','taxiFare_장':'taxiFare_J'},inplace=True)
df

,cartype,예정시간,배차시간,승차시간,startpos1,startpos2,endpos1,endpos2,direction,start_lat,...,duration,taxiFare_N,y_b_time,b_s_time,y_s_time,y_b_seconds,b_s_seconds,y_s_seconds,taxiFare_I,taxiFare_J
0,중형승합,2018-01-23 00:04:00,2018-01-23 00:18:42,2018-01-23 00:34:08,성북구,장위제1동,동대문구,회기동,장위제1동회기동,127.043713,...,648.464,5600,0 days 00:14:42,0 days 00:15:26,0 days 00:30:08,882,926,1808,7700,1500
1,중형승합,2018-01-23 00:17:00,2018-01-23 00:50:37,2018-01-23 01:19:22,중랑구,면목제7동,노원구,상계1동,면목제7동상계1동,127.086985,...,1318.675,13700,0 days 00:33:37,0 days 00:28:45,0 days 01:02:22,2017,1725,3742,16200,3200
2,중형승합,2018-01-23 00:17:06,2018-01-23 00:17:39,2018-01-23 00:30:00,성북구,정릉제3동,강북구,수유제3동,정릉제3동수유제3동,127.004186,...,864.161,6200,0 days 00:00:33,0 days 00:12:21,0 days 00:12:54,33,741,774,8200,1500
3,중형승합,2018-01-23 00:47:00,2018-01-23 00:51:19,2018-01-23 01:09:33,구로구,고척제2동,관악구,은천동,고척제2동은천동,126.858712,...,1652.785,11000,0 days 00:04:19,0 days 00:18:14,0 days 00:22:33,259,1094,1353,12700,2900
4,중형승합,2018-01-23 01:08:00,2018-01-23 01:44:17,2018-01-23 02:06:53,동대문구,회기동,성북구,장위제2동,회기동장위제2동,127.055313,...,598.639,5200,0 days 00:36:17,0 days 00:22:36,0 days 00:58:53,2177,1356,3533,7300,1500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
407548,대형승용,2022-06-24 07:00:00,2022-06-24 07:12:38,2022-06-24 07:55:24,성북구,돈암제2동,강남구,대치4동,돈암제2동대치4동,127.010566,...,3581.405,18900,0 days 00:12:38,0 days 00:42:46,0 days 00:55:24,758,2566,3324,19300,3200
407549,중형승합,2022-06-24 07:00:00,2022-06-24 07:09:10,2022-06-24 07:25:35,은평구,구산동,은평구,구산동,구산동구산동,126.910389,...,240.000,0,0 days 00:09:10,0 days 00:16:25,0 days 00:25:35,550,985,1535,4800,1500
407550,중형승합,2022-06-24 07:00:00,2022-06-24 07:04:27,2022-06-24 07:25:35,노원구,상계8동,영등포구,영등포동,상계8동영등포동,127.051587,...,2667.002,45030,0 days 00:04:27,0 days 00:21:08,0 days 00:25:35,267,1268,1535,51100,5500
407551,중형승합,2022-06-24 06:55:01,2022-06-24 07:00:00,2022-06-24 08:06:22,성북구,길음제1동,강북구,번제1동,길음제1동번제1동,127.019392,...,772.956,6200,0 days 00:04:59,0 days 01:06:22,0 days 01:11:21,299,3982,4281,8400,1500


In [514]:
# csv로 저장합니다.
df.to_csv('master_final_220708_택시요금추가.csv', index= False, encoding='utf-8')